In [11]:
import pandas as pd
import numpy as np
from IPython.display import display
import re

In [2]:
df_calendar = pd.read_csv("datasets/Madrid Airbnb/calendar.csv")

In [3]:
df_calendar

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,167183,2021-04-15,f,$45.00,$45.00,1.0,5.0
1,6369,2021-04-15,t,$60.00,$60.00,1.0,1125.0
2,6369,2021-04-16,t,$60.00,$60.00,1.0,1125.0
3,6369,2021-04-17,t,$60.00,$60.00,1.0,1125.0
4,6369,2021-04-18,t,$60.00,$60.00,1.0,1125.0
...,...,...,...,...,...,...,...
7161251,49103093,2022-04-10,f,$28.00,$35.00,1.0,1125.0
7161252,49103093,2022-04-11,f,$28.00,$35.00,1.0,1125.0
7161253,49103093,2022-04-12,f,$28.00,$35.00,1.0,1125.0
7161254,49103093,2022-04-13,f,$28.00,$35.00,1.0,1125.0


In [4]:
df_calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7161256 entries, 0 to 7161255
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           object 
 4   adjusted_price  object 
 5   minimum_nights  float64
 6   maximum_nights  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 382.5+ MB


In [5]:
set([type(i) for i in df_calendar["adjusted_price"].values])

{float, str}

In [6]:
df_calendar["price"] = [(str(i)).strip("$") for i in df_calendar["price"].values]

In [7]:
df_calendar["price"].unique()

array(['45.00', '60.00', '35.00', ..., '779.00', '824.00', '1,819.00'],
      dtype=object)

In [8]:
df_calendar["price"] = [(str(i)).strip(".00") for i in df_calendar["price"].values]

In [9]:
df_calendar["price"].unique()

array(['45', '6', '35', ..., '779', '824', '1,819'], dtype=object)

In [10]:
df_calendar["price"].apply(lambda x: x.replace('""', "0") if pd.notnull(x) else x)

0          45
1           6
2           6
3           6
4           6
           ..
7161251    28
7161252    28
7161253    28
7161254    28
7161255    28
Name: price, Length: 7161256, dtype: object

In [11]:
df_calendar["price"].apply(lambda x: x.replace(',', '.') if pd.notnull(x) else x)

0          45
1           6
2           6
3           6
4           6
           ..
7161251    28
7161252    28
7161253    28
7161254    28
7161255    28
Name: price, Length: 7161256, dtype: object

In [12]:
df_calendar["price"].apply(lambda x: float(x))

ValueError: could not convert string to float: '1,4'

In [13]:
df_calendar.groupby("listing_id").mean()

,minimum_nights,maximum_nights
listing_id,,
6369,1.0,1125.0
21853,4.0,40.0
23001,15.0,730.0
24805,5.0,1125.0
26825,2.0,1125.0
...,...,...
49185822,30.0,1125.0
49186179,30.0,1125.0
49187258,7.0,1125.0


In [14]:
df_listings = pd.read_csv("datasets/Madrid Airbnb/listings.csv")

In [15]:
df_listings

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,6369,"Rooftop terrace room , ensuite bathroom",13660,Simon,Chamartín,Hispanoamérica,40.45724,-3.67688,Private room,60,1,78,2020-09-20,0.58,1,180
1,21853,Bright and airy room,83531,Abdel,Latina,Cármenes,40.40381,-3.74130,Private room,31,4,33,2018-07-15,0.42,2,364
2,23001,Apartmento Arganzuela- Madrid Rio,82175,Jesus,Arganzuela,Legazpi,40.38840,-3.69511,Entire home/apt,50,15,0,NaN,NaN,7,1
3,24805,Gran Via Studio Madrid,346366726,A,Centro,Universidad,40.42183,-3.70529,Entire home/apt,92,5,10,2020-03-01,0.13,1,72
4,26825,Single Room whith private Bathroom,114340,Agustina,Arganzuela,Legazpi,40.38975,-3.69018,Private room,26,2,149,2020-03-12,1.12,1,365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19613,49185822,Habitación con TV con Netflix en Lavapiés,172011610,Belaid,Centro,Embajadores,40.40756,-3.69937,Private room,23,30,0,NaN,NaN,8,349
19614,49186179,Habitación con TV con Netflix en Gaztambide,172011610,Belaid,Chamberí,Gaztambide,40.43706,-3.71364,Private room,21,30,0,NaN,NaN,8,350
19615,49187258,Habitación con TV con Netflix en Chamberí,172011610,Belaid,Chamberí,Arapiles,40.43857,-3.70715,Private room,22,7,0,NaN,NaN,8,364
19616,49187471,Habitación con TV con Netflix en Goya,172011610,Belaid,Salamanca,Guindalera,40.43027,-3.66759,Private room,19,30,0,NaN,NaN,8,349


In [16]:
df_listings.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [17]:
df_listings_detailed = pd.read_csv("datasets/Madrid Airbnb/listings_detailed.csv")

In [18]:
df_listings_detailed

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,6369,https://www.airbnb.com/rooms/6369,20210413152844,2021-04-15,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,NaN,https://a0.muscache.com/pictures/683224/4cc318...,13660,https://www.airbnb.com/users/show/13660,...,10.0,10.0,10.0,NaN,f,1,0,1,0,0.58
1,21853,https://www.airbnb.com/rooms/21853,20210413152844,2021-04-16,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,https://www.airbnb.com/users/show/83531,...,10.0,8.0,9.0,NaN,f,2,0,2,0,0.42
2,23001,https://www.airbnb.com/rooms/23001,20210413152844,2021-04-16,Apartmento Arganzuela- Madrid Rio,"Apartamento de tres dormitorios dobles, gran s...","Barrio Arganzuela, junto a Madrid Rio, zonas c...",https://a0.muscache.com/pictures/58e6a770-509c...,82175,https://www.airbnb.com/users/show/82175,...,NaN,NaN,NaN,NaN,f,7,6,1,0,NaN
3,24805,https://www.airbnb.com/rooms/24805,20210413152844,2021-04-15,Gran Via Studio Madrid,"Studio located 50 meters from Gran Via, next t...","The area is next to the Gran Via, so people li...",https://a0.muscache.com/pictures/miso/Hosting-...,346366726,https://www.airbnb.com/users/show/346366726,...,10.0,10.0,10.0,NaN,t,1,1,0,0,0.13
4,26825,https://www.airbnb.com/rooms/26825,20210413152844,2021-04-15,Single Room whith private Bathroom,Nice and cozy roon for one person with a priva...,"Es un barrio muy tranquilo, en una zona de Mad...",https://a0.muscache.com/pictures/149358/218d5b...,114340,https://www.airbnb.com/users/show/114340,...,9.0,9.0,9.0,NaN,f,1,0,1,0,1.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19613,49185822,https://www.airbnb.com/rooms/49185822,20210413152844,2021-04-15,Habitación con TV con Netflix en Lavapiés,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/59be7fbb-28b3...,172011610,https://www.airbnb.com/users/show/172011610,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19614,49186179,https://www.airbnb.com/rooms/49186179,20210413152844,2021-04-16,Habitación con TV con Netflix en Gaztambide,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/62d916ba-bea9...,172011610,https://www.airbnb.com/users/show/172011610,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19615,49187258,https://www.airbnb.com/rooms/49187258,20210413152844,2021-04-15,Habitación con TV con Netflix en Chamberí,"*facturas incluidas (gas, electricidad, Intern...",NaN,https://a0.muscache.com/pictures/be000599-44d6...,172011610,https://www.airbnb.com/users/show/172011610,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19616,49187471,https://www.airbnb.com/rooms/49187471,20210413152844,2021-04-15,Habitación con TV con Netflix en Goya,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/ffbecf79-945e...,172011610,https://www.airbnb.com/users/show/172011610,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN


In [19]:
df_listings_detailed.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'description',
       'neighborhood_overview', 'picture_url', 'host_id', 'host_url',
       'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
       'maximum_nights_avg_ntm', 'calendar_upd

In [20]:
df_listings_detailed["price"]

0        $60.00
1        $31.00
2        $50.00
3        $92.00
4        $26.00
          ...  
19613    $23.00
19614    $21.00
19615    $22.00
19616    $19.00
19617    $20.00
Name: price, Length: 19618, dtype: object

In [21]:
df_neighbourh = pd.read_csv("neighbourhoods.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'neighbourhoods.csv'

In [22]:
df_neighbourh

NameError: name 'df_neighbourh' is not defined

In [23]:
df_reviews = pd.read_csv("reviews.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'reviews.csv'

In [24]:
df_reviews

NameError: name 'df_reviews' is not defined

In [25]:
df_reviews_detailed = pd.read_csv("reviews_detailed.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'reviews_detailed.csv'

In [26]:
df_reviews_detailed

NameError: name 'df_reviews_detailed' is not defined

In [27]:
nparr_listings_detailed = np.array(df_listings_detailed["price"])

In [28]:
txt = '1,400.00banana,,platano'
txt = txt.replace(',','')
print(txt)

1400.00bananaplatano


In [29]:
nparr_prices = np.array([float(i[1:].replace(",","")) for i in nparr_listings_detailed])
    
nparr_prices

array([60., 31., 50., ..., 22., 19., 20.])

In [30]:
df_listings_detailed["price"] = nparr_prices

In [31]:
df_listings_detailed["price"]

0        60.0
1        31.0
2        50.0
3        92.0
4        26.0
         ... 
19613    23.0
19614    21.0
19615    22.0
19616    19.0
19617    20.0
Name: price, Length: 19618, dtype: float64

In [32]:
for i in df_listings_detailed.columns:
    print(i)
    print("---"*20)
    print(df_listings_detailed[i])
    print("---"*20)

id
------------------------------------------------------------
0            6369
1           21853
2           23001
3           24805
4           26825
           ...   
19613    49185822
19614    49186179
19615    49187258
19616    49187471
19617    49187791
Name: id, Length: 19618, dtype: int64
------------------------------------------------------------
listing_url
------------------------------------------------------------
0            https://www.airbnb.com/rooms/6369
1           https://www.airbnb.com/rooms/21853
2           https://www.airbnb.com/rooms/23001
3           https://www.airbnb.com/rooms/24805
4           https://www.airbnb.com/rooms/26825
                         ...                  
19613    https://www.airbnb.com/rooms/49185822
19614    https://www.airbnb.com/rooms/49186179
19615    https://www.airbnb.com/rooms/49187258
19616    https://www.airbnb.com/rooms/49187471
19617    https://www.airbnb.com/rooms/49187791
Name: listing_url, Length: 19618, dtype: object
-

In [33]:
from collections import Counter

Counter(df_listings_detailed["host_listings_count"].isnull())

Counter({False: 19091, True: 527})

In [34]:
to_test = ["host_response_time","host_response_rate","host_acceptance_rate","host_is_superhost","host_neighbourhood","host_total_listings_count","host_has_profile_pic","host_identity_verified","neighbourhood_cleansed","neighbourhood_group_cleansed","property_type","room_type","accommodates","bathrooms_text","bedrooms","beds","amenities","price","minimum_nights","maximum_nights","availability_30","availability_60","availability_90","availability_365","number_of_reviews","review_scores_rating","review_scores_accuracy","instant_bookable","reviews_per_month"]




In [35]:
for i in to_test:
    
    print(i)
    print("---"*20)
    print(len(df_listings_detailed[i].unique()))
    display(df_listings_detailed[i].unique())
    display(Counter(df_listings_detailed[i]))
    print("---"*20)


host_response_time
------------------------------------------------------------
5


array(['within an hour', nan, 'a few days or more', 'within a few hours',
       'within a day'], dtype=object)

Counter({'within an hour': 6450,
         nan: 8477,
         'a few days or more': 1340,
         'within a few hours': 1725,
         'within a day': 1626})

------------------------------------------------------------
host_response_rate
------------------------------------------------------------
73


array(['100%', nan, '82%', '33%', '0%', '67%', '77%', '87%', '90%', '86%',
       '96%', '80%', '92%', '60%', '88%', '70%', '17%', '83%', '50%',
       '91%', '30%', '56%', '97%', '8%', '94%', '85%', '75%', '69%',
       '29%', '98%', '74%', '72%', '38%', '93%', '95%', '64%', '40%',
       '78%', '89%', '57%', '20%', '22%', '15%', '71%', '99%', '81%',
       '43%', '14%', '42%', '76%', '13%', '79%', '11%', '10%', '58%',
       '46%', '63%', '23%', '61%', '25%', '51%', '37%', '27%', '39%',
       '5%', '9%', '68%', '59%', '44%', '7%', '18%', '55%', '53%'],
      dtype=object)

...

Counter({'f': 9956, 't': 9662})

------------------------------------------------------------
reviews_per_month
------------------------------------------------------------
683


array([5.800e-01, 4.200e-01,       nan, 1.300e-01, 1.120e+00, 3.000e-02,
       1.290e+00, 1.100e-01, 9.100e-01, 3.800e-01, 4.400e-01, 1.500e-01,
       2.210e+00, 4.600e-01, 3.300e-01, 1.000e-01, 8.700e-01, 5.100e-01,
       8.900e-01, 1.530e+00, 1.280e+00, 1.750e+00, 7.600e-01, 5.310e+00,
       6.400e-01, 5.000e-02, 6.800e-01, 1.900e-01, 1.570e+00, 9.800e-01,
       6.700e-01, 1.680e+00, 1.840e+00, 4.460e+00, 8.600e-01, 6.000e-01,
       8.200e-01, 7.000e-02, 2.200e-01, 1.650e+00, 9.000e-02, 5.200e-01,
       9.300e-01, 2.460e+00, 3.500e-01, 1.270e+00, 4.500e-01, 2.330e+00,
       8.000e-02, 1.660e+00, 1.060e+00, 1.810e+00, 1.600e+00, 1.300e+00,
       3.000e-01, 4.250e+00, 1.320e+00, 3.200e-01, 1.240e+00, 4.220e+00,
       1.050e+00, 3.400e-01, 1.900e+00, 4.900e-01, 6.500e-01, 1.020e+00,
       4.700e-01, 1.030e+00, 2.770e+00, 1.890e+00, 2.500e-01, 5.900e-01,
       1.400e+00, 4.000e-01, 9.600e-01, 1.130e+00, 1.440e+00, 2.000e-02,
       1.510e+00, 1.070e+00, 4.000e-02, 2.800e-01, 

Counter({0.58: 74,
         0.42: 85,
         nan: 1,
         0.13: 189,
         1.12: 29,
         0.03: 216,
         1.29: 23,
         0.11: 172,
         nan: 1,
         0.91: 38,
         0.38: 61,
         0.44: 61,
         0.15: 176,
         2.21: 17,
         0.46: 55,
         0.33: 81,
         0.1: 166,
         0.87: 37,
         0.51: 62,
         0.89: 43,
         1.53: 28,
         1.28: 26,
         1.75: 17,
         0.76: 33,
         5.31: 3,
         0.64: 50,
         0.05: 286,
         0.68: 39,
         nan: 1,
         0.19: 131,
         1.57: 21,
         0.98: 40,
         0.67: 48,
         1.68: 19,
         1.84: 23,
         4.46: 3,
         0.86: 37,
         0.6: 43,
         0.82: 43,
         0.07: 357,
         0.22: 138,
         1.65: 21,
         0.09: 193,
         0.52: 66,
         0.93: 27,
         2.46: 15,
         0.35: 84,
         1.27: 35,
         nan: 1,
         0.45: 88,
         2.33: 21,
         0.08: 163,
         1.66

------------------------------------------------------------


In [36]:
df_listings_detailed[df_listings_detailed["bathrooms_text"] == "1 bath"]["listing_url"]

1           https://www.airbnb.com/rooms/21853
3           https://www.airbnb.com/rooms/24805
6           https://www.airbnb.com/rooms/30320
7           https://www.airbnb.com/rooms/30959
9           https://www.airbnb.com/rooms/40718
                         ...                  
19603    https://www.airbnb.com/rooms/49166394
19604    https://www.airbnb.com/rooms/49167392
19609    https://www.airbnb.com/rooms/49175062
19611    https://www.airbnb.com/rooms/49177673
19612    https://www.airbnb.com/rooms/49184441
Name: listing_url, Length: 8828, dtype: object

In [44]:
df_listings_detailed["bathrooms_text"].replace(np.nan,"?", inplace = True)

In [45]:
l_category = []

for i in df_listings_detailed["bathrooms_text"].values:
    
    print(i)
    if "shared" in i:
        
        l_category.append("Shared")
    
    elif "private" in i:
        
        l_category.append("Private")
        
    else:
        
        l_category.append("Unknown")

print("Shared ones")
print("---"*20)
display(l_shared)
print("Private ones")
print("---"*20)
display(l_private)
print("Unknown ones")
print("---"*20)
display(l_unknown)
        

1 shared bath
1 bath
2 baths
1 bath
1 private bath
?
1 bath
1 bath
?
1 bath
1 bath
2 baths
1 shared bath
1.5 shared baths
1 shared bath
1 bath
1 bath
1.5 baths
1 bath
1 bath
1 bath
1 bath
1 bath
1 bath
1.5 shared baths
2 shared baths
1 bath
1 bath
1 bath
1 shared bath
1 bath
2 baths
1 bath
1 bath
1 shared bath
1 private bath
1 bath
2.5 baths
1 bath
1 shared bath
1 bath
1 bath
1 private bath
1 bath
1 bath
1 bath
1 shared bath
1 shared bath
1 bath
1 shared bath
1 private bath
1 bath
1 shared bath
2 baths
1 bath
1 private bath
1 shared bath
1 bath
1 private bath
1 bath
2 baths
Shared half-bath
1 shared bath
Half-bath
1 shared bath
0 baths
1 bath
1 bath
1.5 baths
1 bath
1 bath
1 bath
1 bath
1 bath
1 bath
1 bath
2 baths
2 baths
1 bath
1 private bath
1 shared bath
1 shared bath
1 bath
1 bath
1 bath
4 baths
1 bath
1 bath
2 baths
1 bath
1.5 baths
2 shared baths
1 bath
1 bath
1 bath
1 bath
1 bath
1 shared bath
1 bath
2 baths
1 private bath
1.5 baths
2 baths
2 baths
1 bath
1 bath
1.5 shared bath

NameError: name 'l_shared' is not defined

In [46]:
nparr_shared = np.array(l_shared)

df_baths = pd.DataFrame()

df_baths["Shared"] = l_shared

len(df_baths["Shared"].unique())

NameError: name 'l_shared' is not defined

In [ ]:
nparr_shared = np.array(l_shared)

df_baths = pd.DataFrame()

df_baths["Private"] = l_private

df_baths["Private"]

In [ ]:
nparr_shared = np.array(l_shared)

df_baths = pd.DataFrame()

df_baths["Unknown"] = l_unknown

len(df_baths["Unknown"].unique())

In [ ]:
for i in (df_listings_detailed["bathrooms_text"].values):
    print(i)

In [ ]:
df_listings_detailed["bathrooms_text"] = df_listings_detailed["bathrooms_text"].replace(np.nan,"?")

In [47]:
import re

l_nums = [re.findall(r'\d+',i) for i in df_listings_detailed["bathrooms_text"].values]

l_nums_completed = []

for i in l_nums:
    
    if len(i) > 1:
        
        l_nums_completed.append('.'.join(i))
    
    elif len(i) == 0:
        
        l_nums_completed.append('0')
        
    else:
        
        l_nums_completed.append(i[0])
        
l_nums_completed
        

['1',
 '1',
 '2',
 '1',
 '1',
 '0',
 '1',
 '1',
 '0',
 '1',
 '1',
 '2',
 '1',
 '1.5',
 '1',
 '1',
 '1',
 '1.5',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.5',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2.5',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '1',
 '1.5',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '4',
 '1',
 '1',
 '2',
 '1',
 '1.5',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1.5',
 '2',
 '2',
 '1',
 '1',
 '1.5',
 '1',
 '1.5',
 '1',
 '1.5',
 '1',
 '1',
 '1.5',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '0',
 '1.5',
 '2',
 '1.5',
 '2',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '1',
 '1',
 '2',
 '1.5',
 '1',
 '2',
 '2',
 '1.5',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1.5',
 '2',
 '1',
 '1.5',
 '1',
 '1.5',
 '1',
 '1',
 '1',
 '1.5',
 '1',
 '1',
 '

In [48]:
df_listings_detailed[to_test].columns

Index(['host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_total_listings_count',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'property_type', 'room_type', 'accommodates', 'bathrooms_text',
       'bedrooms', 'beds', 'amenities', 'price', 'minimum_nights',
       'maximum_nights', 'availability_30', 'availability_60',
       'availability_90', 'availability_365', 'number_of_reviews',
       'review_scores_rating', 'review_scores_accuracy', 'instant_bookable',
       'reviews_per_month'],
      dtype='object')

In [49]:
l_to_test = ["host_is_superhost","neighbourhood_cleansed","neighbourhood_group_cleansed","property_type","room_type","accommodates","bathrooms_text","beds","price","minimum_nights","maximum_nights","availability_30","availability_365","number_of_reviews","instant_bookable"]
len(l_to_test)

15

In [50]:
df_listings_detailed["property_type"]

0        Private room in apartment
1        Private room in apartment
2                 Entire apartment
3                 Entire apartment
4            Private room in house
                   ...            
19613    Private room in apartment
19614    Private room in apartment
19615    Private room in apartment
19616    Private room in apartment
19617    Private room in apartment
Name: property_type, Length: 19618, dtype: object

In [51]:
df_listings_detailed[["availability_30","availability_60","availability_90","availability_365"]].corr()

,availability_30,availability_60,availability_90,availability_365
availability_30,1.000000,0.968983,0.931023,0.610209
availability_60,0.968983,1.000000,0.984885,0.667003
availability_90,0.931023,0.984885,1.000000,0.709008
availability_365,0.610209,0.667003,0.709008,1.000000


In [52]:
l_to_test = ["host_is_superhost","neighbourhood_cleansed","neighbourhood_group_cleansed","property_type","room_type","accommodates","bathrooms_text","beds","price","minimum_nights","maximum_nights","availability_30","availability_365","number_of_reviews","instant_bookable"]

df_airbnb = df_listings_detailed[l_to_test]

In [53]:
df_airbnb

,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,bathrooms_text,beds,price,minimum_nights,maximum_nights,availability_30,availability_365,number_of_reviews,instant_bookable
0,t,Hispanoamérica,Chamartín,Private room in apartment,Private room,2,1 shared bath,1.0,60.0,1,1125,30,180,78,f
1,f,Cármenes,Latina,Private room in apartment,Private room,1,1 bath,1.0,31.0,4,40,29,364,33,f
2,f,Legazpi,Arganzuela,Entire apartment,Entire home/apt,6,2 baths,5.0,50.0,15,730,1,1,0,f
3,f,Universidad,Centro,Entire apartment,Entire home/apt,3,1 bath,1.0,92.0,5,730,12,72,10,t
4,f,Legazpi,Arganzuela,Private room in house,Private room,1,1 private bath,1.0,26.0,2,1125,30,365,149,f
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19613,f,Embajadores,Centro,Private room in apartment,Private room,1,2 shared baths,1.0,23.0,30,1125,14,349,0,f
19614,f,Gaztambide,Chamberí,Private room in apartment,Private room,1,3.5 shared baths,1.0,21.0,30,1125,15,350,0,f
19615,f,Arapiles,Chamberí,Private room in apartment,Private room,1,3 baths,1.0,22.0,7,1125,29,364,0,f
19616,f,Guindalera,Salamanca,Private room in apartment,Private room,1,3 shared baths,1.0,19.0,30,1125,14,349,0,f


In [5]:
class airbnb_city:
    
    def __init__(self, csv):
        
        self.csv = csv
                
        self.df_city = pd.read_csv(self.csv)
        
        print("Instance created!")
        
    def clean_columns(self):
        
        # Take only the relevant columns in the dataframe
        
        l_relevant_columns = ["host_is_superhost","neighbourhood_cleansed","neighbourhood_group_cleansed","property_type","room_type","accommodates","bathrooms_text","beds","price","minimum_nights","maximum_nights","availability_30","availability_365","number_of_reviews","instant_bookable"]

        self.df_city = self.df_city[l_relevant_columns]
        
        self.df_city["bathrooms_text"].replace(np.nan, "?", inplace = True)
        
        # Get numbers out of bathroom_text columns
        
        l_nums = [re.findall(r'\d+',i) for i in self.df_city["bathrooms_text"].values]

        l_nums_completed = []

        for i in l_nums:

            if len(i) > 1:

                l_nums_completed.append('.'.join(i))

            elif len(i) == 0:

                l_nums_completed.append('0')

            else:

                l_nums_completed.append(i[0])

        # Separate categories from bathroom_text
        
        l_category = []

        for i in self.df_city["bathrooms_text"].values:

            if "shared" in i:

                l_category.append("Shared")

            elif "private" in i:

                l_category.append("Private")

            else:

                l_category.append("Unknown")
                
        # Create two different columns replacing bathroom_text
        
        self.df_city.drop("bathrooms_text", axis = 1, inplace = True)
                
        self.df_city["num_of_baths"] = l_nums_completed
        
        self.df_city["bath_category"] = l_category

        # Column["prices"]

        # Replacing ","
        self.df_city["price"]  = self.df_city["price"].apply(lambda x: float(x.strip("$").replace(",","") if pd.notnull(x) else x))

        print("Dataframe sucessfully created!")

        
    def return_df(self):
    
        return self.df_city
    
    def display_df(self):
    
        display(self.df_city)

In [9]:
madrid = airbnb_city("datasets/Airbnb/madrid.csv")
valencia = airbnb_city("datasets/Airbnb/valencia.csv")
sevilla = airbnb_city("datasets/Airbnb/sevilla.csv")

Instance created!
Instance created!
Instance created!


In [12]:
madrid.clean_columns()
valencia.clean_columns()
sevilla.clean_columns()

Dataframe sucessfully created!
Dataframe sucessfully created!
Dataframe sucessfully created!


In [13]:
madrid.display_df()
valencia.display_df()
sevilla.display_df()

,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,beds,price,minimum_nights,maximum_nights,availability_30,availability_365,number_of_reviews,instant_bookable,num_of_baths,bath_category
0,f,Hispanoamérica,Chamartín,Private room in rental unit,Private room,2,1.0,77.0,3,1125,0,56,99,f,1,Private
1,f,Cármenes,Latina,Private room in rental unit,Private room,1,1.0,31.0,4,40,0,255,33,f,1,Unknown
2,f,Universidad,Centro,Entire rental unit,Entire home/apt,3,1.0,92.0,5,1125,1,80,24,f,1,Unknown
3,f,Legazpi,Arganzuela,Private room in home,Private room,1,1.0,26.0,2,1125,12,339,166,f,1,Private
4,t,Sol,Centro,Private room in rental unit,Private room,2,1.0,49.0,2,90,2,271,475,f,1.5,Shared
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20676,f,San Andrés,Villaverde,Entire rental unit,Entire home/apt,5,3.0,85.0,1,1125,0,0,1,f,1,Unknown
20677,f,San Andrés,Villaverde,Entire rental unit,Entire home/apt,6,4.0,75.0,7,1125,0,224,3,f,1,Unknown
20678,t,San Andrés,Villaverde,Private room in home,Private room,1,1.0,50.0,1,1125,30,365,18,f,1,Private
20679,t,Aravaca,Moncloa - Aravaca,Entire condo,Entire home/apt,4,3.0,165.0,4,125,23,173,12,f,1.5,Unknown


,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,beds,price,minimum_nights,maximum_nights,availability_30,availability_365,number_of_reviews,instant_bookable,num_of_baths,bath_category
0,f,LA SEU,CIUTAT VELLA,Entire rental unit,Entire home/apt,3,1.0,100.0,1,730,12,138,1,f,0,Unknown
1,f,EL CARME,CIUTAT VELLA,Entire rental unit,Entire home/apt,6,6.0,156.0,2,1125,8,182,108,f,2,Unknown
2,f,ARRANCAPINS,EXTRAMURS,Entire rental unit,Entire home/apt,6,3.0,141.0,3,66,0,0,135,f,2,Unknown
3,f,LA MALVA-ROSA,POBLATS MARITIMS,Entire villa,Entire home/apt,15,20.0,396.0,1,1125,1,284,168,f,5.5,Unknown
4,f,EL PILAR,CIUTAT VELLA,Entire serviced apartment,Entire home/apt,5,3.0,94.0,2,1125,5,181,16,f,1,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7350,t,MASSARROJOS,POBLATS DEL NORD,Entire chalet,Entire home/apt,8,5.0,250.0,7,28,0,0,3,f,3.5,Unknown
7351,f,BENIMAMET,POBLATS DE L'OEST,Entire rental unit,Entire home/apt,4,3.0,58.0,3,40,0,0,20,f,2,Unknown
7352,f,MASSARROJOS,POBLATS DEL NORD,Entire rental unit,Entire home/apt,13,14.0,480.0,5,365,0,0,0,t,4,Unknown
7353,t,POBLE NOU,POBLATS DEL NORD,Entire rental unit,Entire home/apt,7,5.0,165.0,4,1125,0,0,7,t,4,Unknown


,host_is_superhost,neighbourhood_cleansed,neighbourhood_group_cleansed,property_type,room_type,accommodates,beds,price,minimum_nights,maximum_nights,availability_30,availability_365,number_of_reviews,instant_bookable,num_of_baths,bath_category
0,f,San Vicente,Casco Antiguo,Entire rental unit,Entire home/apt,4,2.0,99.0,2,120,0,117,168,t,1,Unknown
1,f,San Lorenzo,Casco Antiguo,Entire rental unit,Entire home/apt,3,1.0,75.0,3,1125,6,241,42,t,1,Unknown
2,f,San Bartolomé,Casco Antiguo,Private room in home,Private room,2,1.0,79.0,2,365,25,360,86,f,1,Shared
3,t,San Lorenzo,Casco Antiguo,Entire rental unit,Entire home/apt,6,3.0,84.0,2,360,15,44,160,f,1,Unknown
4,f,San Lorenzo,Casco Antiguo,Entire rental unit,Entire home/apt,6,4.0,85.0,3,365,0,119,103,t,1,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6489,f,Triana Casco Antiguo,Triana,Entire rental unit,Entire home/apt,2,1.0,56.0,2,365,10,160,0,f,1,Unknown
6490,f,San Julián,Casco Antiguo,Private room in rental unit,Private room,2,2.0,60.0,1,365,10,211,0,t,1,Shared
6491,f,Triana Casco Antiguo,Triana,Entire rental unit,Entire home/apt,4,3.0,83.0,1,365,27,83,0,t,1,Unknown
6492,f,Los Pajaros,Cerro - Amate,Private room in rental unit,Private room,1,1.0,32.0,1,365,30,365,0,t,1,Shared


In [60]:
df_listings_detailed.drop("id", axis = 1)

,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,https://www.airbnb.com/rooms/6369,20210413152844,2021-04-15,"Rooftop terrace room , ensuite bathroom",Excellent connection with the AIRPORT and EXHI...,NaN,https://a0.muscache.com/pictures/683224/4cc318...,13660,https://www.airbnb.com/users/show/13660,Simon,...,10.0,10.0,10.0,NaN,f,1,0,1,0,0.58
1,https://www.airbnb.com/rooms/21853,20210413152844,2021-04-16,Bright and airy room,We have a quiet and sunny room with a good vie...,We live in a leafy neighbourhood with plenty o...,https://a0.muscache.com/pictures/68483181/87bc...,83531,https://www.airbnb.com/users/show/83531,Abdel,...,10.0,8.0,9.0,NaN,f,2,0,2,0,0.42
2,https://www.airbnb.com/rooms/23001,20210413152844,2021-04-16,Apartmento Arganzuela- Madrid Rio,"Apartamento de tres dormitorios dobles, gran s...","Barrio Arganzuela, junto a Madrid Rio, zonas c...",https://a0.muscache.com/pictures/58e6a770-509c...,82175,https://www.airbnb.com/users/show/82175,Jesus,...,NaN,NaN,NaN,NaN,f,7,6,1,0,NaN
3,https://www.airbnb.com/rooms/24805,20210413152844,2021-04-15,Gran Via Studio Madrid,"Studio located 50 meters from Gran Via, next t...","The area is next to the Gran Via, so people li...",https://a0.muscache.com/pictures/miso/Hosting-...,346366726,https://www.airbnb.com/users/show/346366726,A,...,10.0,10.0,10.0,NaN,t,1,1,0,0,0.13
4,https://www.airbnb.com/rooms/26825,20210413152844,2021-04-15,Single Room whith private Bathroom,Nice and cozy roon for one person with a priva...,"Es un barrio muy tranquilo, en una zona de Mad...",https://a0.muscache.com/pictures/149358/218d5b...,114340,https://www.airbnb.com/users/show/114340,Agustina,...,9.0,9.0,9.0,NaN,f,1,0,1,0,1.12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19613,https://www.airbnb.com/rooms/49185822,20210413152844,2021-04-15,Habitación con TV con Netflix en Lavapiés,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/59be7fbb-28b3...,172011610,https://www.airbnb.com/users/show/172011610,Belaid,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19614,https://www.airbnb.com/rooms/49186179,20210413152844,2021-04-16,Habitación con TV con Netflix en Gaztambide,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/62d916ba-bea9...,172011610,https://www.airbnb.com/users/show/172011610,Belaid,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19615,https://www.airbnb.com/rooms/49187258,20210413152844,2021-04-15,Habitación con TV con Netflix en Chamberí,"*facturas incluidas (gas, electricidad, Intern...",NaN,https://a0.muscache.com/pictures/be000599-44d6...,172011610,https://www.airbnb.com/users/show/172011610,Belaid,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
19616,https://www.airbnb.com/rooms/49187471,20210413152844,2021-04-15,Habitación con TV con Netflix en Goya,"*facturas incluidas (gas, electricidad, intern...",NaN,https://a0.muscache.com/pictures/ffbecf79-945e...,172011610,https://www.airbnb.com/users/show/172011610,Belaid,...,NaN,NaN,NaN,NaN,f,8,0,8,0,NaN
